In [7]:
import pandas as pd
import numpy as np
from struct import unpack
from base64 import b64decode
import string

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, concatenate, BatchNormalization, Multiply, Add
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

from functools import partial
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, Multiply, Add, Lambda, Layer
from tensorflow.keras.initializers import Constant
from sklearn.model_selection import train_test_split

In [3]:
filename = "/Users/dmitry/Downloads/topics_dataset.json"
df = pd.read_json(filename, lines=True)

In [4]:
IMG_LEN = 1024
TXT_LEN = 300
N_CLASSES = 50

In [5]:
def unpck(l, x):
    return unpack('%df' % l, b64decode(x.encode('utf-8')))

unpck_img = partial(unpck, IMG_LEN)
unpck_txt = partial(unpck, TXT_LEN)

In [6]:
df_q = df.sample(frac=0.25)
x_img_q = np.stack(df_q['x1'].map(unpck_img), axis=0)
x_txt_q = np.stack(df_q['x2'].map(unpck_txt), axis=0)
y_q = to_categorical(np.array(df_q['y1']), N_CLASSES)

In [8]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_train, y_test = train_test_split(x_img_q, x_txt_q, y_q, test_size=0.2, random_state=42)

In [19]:
def get_simple_autoencoder():
    inp_img = Input(shape=(IMG_LEN,))
    inp_txt = Input(shape=(TXT_LEN,))

    concat = concatenate([inp_img, inp_txt])
    encoded = Dense(512, activation='relu')(concat)
    
    out_img = Dense(IMG_LEN)(encoded)
    out_txt = Dense(TXT_LEN)(encoded)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse')
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=30, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=encoded)

In [20]:
encoder = get_simple_autoencoder()

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 18s 482us/sample - loss: 0.7945 - dense_12_loss: 0.6168 - dense_13_loss: 0.1777 - val_loss: 0.6616 - val_dense_12_loss: 0.5618 - val_dense_13_loss: 0.1000
Epoch 2/30
38221/38221 [==============================] - 19s 500us/sample - loss: 0.6184 - dense_12_loss: 0.5411 - dense_13_loss: 0.0772 - val_loss: 0.5908 - val_dense_12_loss: 0.5281 - val_dense_13_loss: 0.0628
Epoch 3/30
38221/38221 [==============================] - 16s 429us/sample - loss: 0.5740 - dense_12_loss: 0.5182 - dense_13_loss: 0.0559 - val_loss: 0.5616 - val_dense_12_loss: 0.5103 - val_dense_13_loss: 0.0514
Epoch 4/30
38221/38221 [==============================] - 18s 484us/sample - loss: 0.5501 - dense_12_loss: 0.5008 - dense_13_loss: 0.0493 - val_loss: 0.5404 - val_dense_12_loss: 0.4920 - val_dense_13_loss: 0.0485
Epoch 5/30
38221/38221 [==============================] - 19s 487us/sample - loss: 0.5288 - dense_1

In [35]:
def get_autoencoder_1():
    inp_img = Input(shape=(IMG_LEN,))
    inp_txt = Input(shape=(TXT_LEN,))

    concat = concatenate([inp_img, inp_txt])
    encoded = Dense(512, activation='relu')(concat)
    
    out_img = Dense(IMG_LEN, activation='sigmoid', name='img')(encoded)
    out_txt = Dense(TXT_LEN, activation='sigmoid', name='txt')(encoded)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse', loss_weights={'img':1, 'txt':0.2})
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=30, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=encoded)

In [36]:
encoder_1 = get_autoencoder_1()

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 23s 609us/sample - loss: 0.3775 - img_loss: 0.3235 - txt_loss: 0.2698 - val_loss: 0.3755 - val_img_loss: 0.3225 - val_txt_loss: 0.2652
Epoch 2/30
38221/38221 [==============================] - 18s 474us/sample - loss: 0.3737 - img_loss: 0.3216 - txt_loss: 0.2610 - val_loss: 0.3720 - val_img_loss: 0.3207 - val_txt_loss: 0.2569
Epoch 3/30
38221/38221 [==============================] - 17s 449us/sample - loss: 0.3706 - img_loss: 0.3199 - txt_loss: 0.2532 - val_loss: 0.3691 - val_img_loss: 0.3192 - val_txt_loss: 0.2496
Epoch 4/30
38221/38221 [==============================] - 22s 577us/sample - loss: 0.3678 - img_loss: 0.3186 - txt_loss: 0.2462 - val_loss: 0.3665 - val_img_loss: 0.3180 - val_txt_loss: 0.2430
Epoch 5/30
38221/38221 [==============================] - 19s 497us/sample - loss: 0.3654 - img_loss: 0.3174 - txt_loss: 0.2399 - val_loss: 0.3642 - val_img_loss: 0.3169 - val_txt

In [21]:
print(encoder.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 1024)]       0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 300)]        0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 1324)         0           input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 512)          678400      concatenate_4[0][0]        

In [24]:
train_e = encoder.predict([x_img_train, x_txt_train])
test_e = encoder.predict([x_img_test, x_txt_test])

In [28]:
np.shape(train_e)

(42468, 512)

In [31]:
def get_model():
    inp = Input(shape=(512,))
    x = Dense(512, activation='relu')(inp)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(50, activation='softmax')(x)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [32]:
model = get_model()
model.fit(train_e, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 6s 164us/sample - loss: 3.4458 - accuracy: 0.1243 - val_loss: 3.0151 - val_accuracy: 0.2437
Epoch 2/20
38221/38221 [==============================] - 6s 150us/sample - loss: 3.0504 - accuracy: 0.2208 - val_loss: 2.8126 - val_accuracy: 0.2861
Epoch 3/20
38221/38221 [==============================] - 6s 146us/sample - loss: 2.9124 - accuracy: 0.2537 - val_loss: 2.7067 - val_accuracy: 0.3063
Epoch 4/20
38221/38221 [==============================] - 6s 152us/sample - loss: 2.8222 - accuracy: 0.2709 - val_loss: 2.6727 - val_accuracy: 0.3193
Epoch 5/20
38221/38221 [==============================] - 6s 151us/sample - loss: 2.7781 - accuracy: 0.2847 - val_loss: 2.6448 - val_accuracy: 0.3153
Epoch 6/20
38221/38221 [==============================] - 6s 148us/sample - loss: 2.7433 - accuracy: 0.2882 - val_loss: 2.5969 - val_accuracy: 0.3308
Epoch 7/20
38221/38221 [===========================

In [38]:
train_e_1 = encoder_1.predict([x_img_train, x_txt_train])
model_1 = get_model()
model_1.fit(train_e_1, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 7s 182us/sample - loss: 3.0136 - accuracy: 0.2319 - val_loss: 2.5085 - val_accuracy: 0.3551
Epoch 2/20
38221/38221 [==============================] - 6s 160us/sample - loss: 2.6394 - accuracy: 0.3188 - val_loss: 2.3875 - val_accuracy: 0.3932
Epoch 3/20
38221/38221 [==============================] - 6s 160us/sample - loss: 2.5354 - accuracy: 0.3435 - val_loss: 2.3025 - val_accuracy: 0.3993
Epoch 4/20
38221/38221 [==============================] - 7s 179us/sample - loss: 2.4800 - accuracy: 0.3547 - val_loss: 2.2831 - val_accuracy: 0.4125
Epoch 5/20
38221/38221 [==============================] - 7s 195us/sample - loss: 2.4458 - accuracy: 0.3641 - val_loss: 2.2725 - val_accuracy: 0.4045
Epoch 6/20
38221/38221 [==============================] - 7s 185us/sample - loss: 2.4075 - accuracy: 0.3721 - val_loss: 2.2590 - val_accuracy: 0.4125
Epoch 7/20
38221/38221 [===========================

In [44]:
def get_autoencoder_2():
    inp_img = Input(shape=(IMG_LEN,))
    inp_txt = Input(shape=(TXT_LEN,))

    concat = concatenate([inp_img, inp_txt])
    hidden_1 = Dense(512, activation='relu')(concat)
    
    hidden_2 = Dropout(0.5)(hidden_1)
    hidden_2 = Dense(512, activation='relu')(hidden_2)
    
    hidden_3 = Dropout(0.5)(hidden_2)
    hidden_3 = Dense(512, activation='relu')(hidden_3)
    
    out_img = Dense(IMG_LEN, activation='sigmoid', name='img')(hidden_3)
    out_txt = Dense(TXT_LEN, activation='sigmoid', name='txt')(hidden_3)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse', loss_weights={'img':1, 'txt':0.2})
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=30, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=hidden_2)

In [45]:
encoder_2 = get_autoencoder_2()

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 52s 1ms/sample - loss: 0.3781 - img_loss: 0.3244 - txt_loss: 0.2686 - val_loss: 0.3659 - val_img_loss: 0.3156 - val_txt_loss: 0.2517
Epoch 2/30
38221/38221 [==============================] - 39s 1ms/sample - loss: 0.3750 - img_loss: 0.3225 - txt_loss: 0.2626 - val_loss: 0.3644 - val_img_loss: 0.3148 - val_txt_loss: 0.2484
Epoch 3/30
38221/38221 [==============================] - 57s 1ms/sample - loss: 0.3725 - img_loss: 0.3210 - txt_loss: 0.2577 - val_loss: 0.3633 - val_img_loss: 0.3142 - val_txt_loss: 0.2458
Epoch 4/30
38221/38221 [==============================] - 45s 1ms/sample - loss: 0.3706 - img_loss: 0.3199 - txt_loss: 0.2537 - val_loss: 0.3624 - val_img_loss: 0.3137 - val_txt_loss: 0.2437
Epoch 5/30
38221/38221 [==============================] - 38s 1000us/sample - loss: 0.3690 - img_loss: 0.3189 - txt_loss: 0.2503 - val_loss: 0.3617 - val_img_loss: 0.3133 - val_txt_loss: 

In [46]:
train_e_2 = encoder_2.predict([x_img_train, x_txt_train])
model_2 = get_model()
model_2.fit(train_e_2, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 8s 211us/sample - loss: 2.9431 - accuracy: 0.2522 - val_loss: 2.4769 - val_accuracy: 0.3600
Epoch 2/20
38221/38221 [==============================] - 7s 177us/sample - loss: 2.5965 - accuracy: 0.3273 - val_loss: 2.3708 - val_accuracy: 0.3866
Epoch 3/20
38221/38221 [==============================] - 7s 188us/sample - loss: 2.4999 - accuracy: 0.3506 - val_loss: 2.3067 - val_accuracy: 0.3998
Epoch 4/20
38221/38221 [==============================] - 7s 182us/sample - loss: 2.4446 - accuracy: 0.3643 - val_loss: 2.3007 - val_accuracy: 0.3991
Epoch 5/20
38221/38221 [==============================] - 7s 176us/sample - loss: 2.4009 - accuracy: 0.3729 - val_loss: 2.2868 - val_accuracy: 0.4055
Epoch 6/20
38221/38221 [==============================] - 7s 189us/sample - loss: 2.3700 - accuracy: 0.3805 - val_loss: 2.2625 - val_accuracy: 0.4052
Epoch 7/20
38221/38221 [===========================

In [43]:
print(encoder_2.summary())

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 1024)]       0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 300)]        0                                            
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 1324)         0           input_23[0][0]                   
                                                                 input_24[0][0]                   
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 512)          678400      concatenate_8[0][0]       

In [59]:
def get_autoencoder_2():
    inp_img = Input(shape=(IMG_LEN,))
    encoded_img = Dense(1024)(inp_img)
    encoded_img = Dropout(0.5)(encoded_img)
    
    inp_txt = Input(shape=(TXT_LEN,))
    encoded_txt = Dense(512)(inp_txt)
    encoded_txt = Dropout(0.5)(encoded_txt)
    
    concat = concatenate([inp_img, inp_txt])
    encoded = Dense(512, activation='relu')(concat)
    
    decoded_img = Dense(1024)(encoded)
    decoded_img = Dropout(0.5)(decoded_img)
    out_img = Dense(IMG_LEN, activation='sigmoid', name='img')(decoded_img)
    
    decoded_txt = Dense(512)(encoded)
    decoded_txt = Dropout(0.5)(decoded_txt)
    out_txt = Dense(TXT_LEN, activation='sigmoid', name='txt')(decoded_img)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse', loss_weights={'img':1, 'txt':0.2})
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=30, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=encoded)

In [60]:
encoder_2 = get_autoencoder_2()

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 36s 935us/sample - loss: 0.3858 - img_loss: 0.3312 - txt_loss: 0.2729 - val_loss: 0.3714 - val_img_loss: 0.3207 - val_txt_loss: 0.2536
Epoch 2/30
38221/38221 [==============================] - 36s 939us/sample - loss: 0.3795 - img_loss: 0.3275 - txt_loss: 0.2601 - val_loss: 0.3668 - val_img_loss: 0.3183 - val_txt_loss: 0.2429
Epoch 3/30
38221/38221 [==============================] - 33s 863us/sample - loss: 0.3747 - img_loss: 0.3247 - txt_loss: 0.2501 - val_loss: 0.3632 - val_img_loss: 0.3164 - val_txt_loss: 0.2344
Epoch 4/30
38221/38221 [==============================] - 35s 911us/sample - loss: 0.3711 - img_loss: 0.3227 - txt_loss: 0.2420 - val_loss: 0.3603 - val_img_loss: 0.3149 - val_txt_loss: 0.2273
Epoch 5/30
38221/38221 [==============================] - 40s 1ms/sample - loss: 0.3681 - img_loss: 0.3210 - txt_loss: 0.2351 - val_loss: 0.3579 - val_img_loss: 0.3137 - val_txt_l

In [61]:
train_e_2 = encoder_2.predict([x_img_train, x_txt_train])
model_2 = get_model()
model_2.fit(train_e_2, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 6s 169us/sample - loss: 2.8997 - accuracy: 0.2604 - val_loss: 2.3908 - val_accuracy: 0.3781
Epoch 2/20
38221/38221 [==============================] - 6s 167us/sample - loss: 2.5550 - accuracy: 0.3387 - val_loss: 2.3074 - val_accuracy: 0.3996
Epoch 3/20
38221/38221 [==============================] - 6s 156us/sample - loss: 2.4606 - accuracy: 0.3610 - val_loss: 2.2601 - val_accuracy: 0.4090
Epoch 4/20
38221/38221 [==============================] - 6s 167us/sample - loss: 2.3999 - accuracy: 0.3778 - val_loss: 2.2374 - val_accuracy: 0.4123
Epoch 5/20
38221/38221 [==============================] - 6s 161us/sample - loss: 2.3633 - accuracy: 0.3821 - val_loss: 2.2136 - val_accuracy: 0.4194
Epoch 6/20
38221/38221 [==============================] - 6s 158us/sample - loss: 2.3262 - accuracy: 0.3947 - val_loss: 2.1877 - val_accuracy: 0.4227
Epoch 7/20
38221/38221 [===========================

In [68]:
model_12 = get_model()
model_12.fit(train_e_2, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 7s 173us/sample - loss: 2.9016 - accuracy: 0.2591 - val_loss: 2.4447 - val_accuracy: 0.3574
Epoch 2/20
38221/38221 [==============================] - 6s 161us/sample - loss: 2.5675 - accuracy: 0.3376 - val_loss: 2.3212 - val_accuracy: 0.3935
Epoch 3/20
38221/38221 [==============================] - 6s 152us/sample - loss: 2.4732 - accuracy: 0.3575 - val_loss: 2.2811 - val_accuracy: 0.4048
Epoch 4/20
38221/38221 [==============================] - 6s 162us/sample - loss: 2.4080 - accuracy: 0.3750 - val_loss: 2.2329 - val_accuracy: 0.4158
Epoch 5/20
38221/38221 [==============================] - 6s 154us/sample - loss: 2.3624 - accuracy: 0.3838 - val_loss: 2.2076 - val_accuracy: 0.4168
Epoch 6/20
38221/38221 [==============================] - 6s 161us/sample - loss: 2.3314 - accuracy: 0.3911 - val_loss: 2.1982 - val_accuracy: 0.4198
Epoch 7/20
38221/38221 [===========================

In [82]:
max(x_txt_train[500])

0.1938919574022293

отскалить данные (текст меньше картинки в 10 раз)

на выход декодера relu/prelu

еще раз попробовать удлинить модель

Проверить данные по отдельности вместе с highway и residual

leaky relu

использовать tensorboard

In [84]:
def get_autoencoder_4():
    inp_img = Input(shape=(IMG_LEN,))
    inp_txt = Input(shape=(TXT_LEN,))

    concat = concatenate([inp_img, inp_txt])
    encoded = Dense(512, activation='relu')(concat)
    
    out_img = Dense(IMG_LEN, activation='sigmoid', name='img')(encoded)
    out_txt = Dense(TXT_LEN, activation='sigmoid', name='txt')(encoded)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse', loss_weights={'img':2, 'txt':0.2})
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=30, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=encoded)

In [85]:
encoder_4 = get_autoencoder_4()

Train on 38221 samples, validate on 4247 samples
Epoch 1/30
38221/38221 [==============================] - 28s 726us/sample - loss: 0.6966 - img_loss: 0.3215 - txt_loss: 0.2680 - val_loss: 0.6926 - val_img_loss: 0.3200 - val_txt_loss: 0.2633
Epoch 2/30
38221/38221 [==============================] - 24s 623us/sample - loss: 0.6888 - img_loss: 0.3185 - txt_loss: 0.2592 - val_loss: 0.6857 - val_img_loss: 0.3174 - val_txt_loss: 0.2554
Epoch 3/30
38221/38221 [==============================] - 21s 540us/sample - loss: 0.6827 - img_loss: 0.3161 - txt_loss: 0.2520 - val_loss: 0.6802 - val_img_loss: 0.3153 - val_txt_loss: 0.2487
Epoch 4/30
38221/38221 [==============================] - 23s 591us/sample - loss: 0.6776 - img_loss: 0.3142 - txt_loss: 0.2457 - val_loss: 0.6755 - val_img_loss: 0.3135 - val_txt_loss: 0.2428
Epoch 5/30
38221/38221 [==============================] - 18s 481us/sample - loss: 0.6732 - img_loss: 0.3126 - txt_loss: 0.2400 - val_loss: 0.6713 - val_img_loss: 0.3120 - val_txt

In [125]:
def get_relu_encoder():
    inp_img = Input(shape=(IMG_LEN,))
    inp_txt = Input(shape=(TXT_LEN,))
    
    encoded = concatenate([inp_img, inp_txt])
    encoded = Dense(512, activation='relu')(inp_img)
    
    out_img = Dense(IMG_LEN, activation='relu', name='img')(encoded)
    out_txt = Dense(TXT_LEN, activation='sigmoid', name='txt')(encoded)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse')
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=10, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=encoded)

In [135]:
def get_relu_encoder_1():
    inp_img = Input(shape=(IMG_LEN,))
    encoded_img = Dense(1024, activation='relu')(inp_img)
#     encoded_img = Dropout(0.5)(encoded_img)
    
    inp_txt = Input(shape=(TXT_LEN,))
    encoded_txt = Dense(512, activation='relu')(inp_txt)
#     encoded_txt = Dropout(0.5)(encoded_txt)
    
    encoded = concatenate([encoded_img, encoded_txt])
    encoded = Dense(512, activation='relu')(inp_img)
    
    decoded_img = Dense(1024, activation='relu')(encoded)
#     decoded_img = Dropout(0.5)(decoded_img)
    
    decoded_txt = Dense(512, activation='relu')(encoded)
#     decoded_txt = Dropout(0.5)(decoded_txt)
    
    out_img = Dense(IMG_LEN, activation='relu', name='img')(decoded_img)
    out_txt = Dense(TXT_LEN, activation='sigmoid', name='txt')(decoded_txt)
    
    autoencoder = Model(inputs=[inp_img, inp_txt], outputs=[out_img, out_txt])
    autoencoder.compile(optimizer='adadelta', loss='mse')
    
    autoencoder.fit([x_img_train, x_txt_train], [x_img_train, x_txt_train], epochs=3, validation_split=0.1)
    return Model(inputs=[inp_img, inp_txt], outputs=encoded)

In [126]:
img_encoder = get_relu_encoder()

Train on 38221 samples, validate on 4247 samples
Epoch 1/10
38221/38221 [==============================] - 18s 483us/sample - loss: 0.0303 - img_loss: 0.0156 - txt_loss: 0.0147 - val_loss: 0.0304 - val_img_loss: 0.0156 - val_txt_loss: 0.0148
Epoch 2/10
38221/38221 [==============================] - 28s 729us/sample - loss: 0.0303 - img_loss: 0.0156 - txt_loss: 0.0147 - val_loss: 0.0303 - val_img_loss: 0.0156 - val_txt_loss: 0.0147
Epoch 3/10
38221/38221 [==============================] - 25s 641us/sample - loss: 0.0302 - img_loss: 0.0156 - txt_loss: 0.0147 - val_loss: 0.0303 - val_img_loss: 0.0156 - val_txt_loss: 0.0147
Epoch 4/10
38221/38221 [==============================] - 27s 713us/sample - loss: 0.0302 - img_loss: 0.0156 - txt_loss: 0.0147 - val_loss: 0.0303 - val_img_loss: 0.0155 - val_txt_loss: 0.0147
Epoch 5/10
38221/38221 [==============================] - 27s 696us/sample - loss: 0.0302 - img_loss: 0.0155 - txt_loss: 0.0146 - val_loss: 0.0302 - val_img_loss: 0.0155 - val_txt

In [136]:
relu_encoder_1 = get_relu_encoder_1()

Train on 38221 samples, validate on 4247 samples
Epoch 1/3
38221/38221 [==============================] - 40s 1ms/sample - loss: 0.0301 - img_loss: 0.0158 - txt_loss: 0.0142 - val_loss: 0.0301 - val_img_loss: 0.0158 - val_txt_loss: 0.0143
Epoch 2/3
38221/38221 [==============================] - 43s 1ms/sample - loss: 0.0300 - img_loss: 0.0158 - txt_loss: 0.0142 - val_loss: 0.0301 - val_img_loss: 0.0158 - val_txt_loss: 0.0143
Epoch 3/3
38221/38221 [==============================] - 44s 1ms/sample - loss: 0.0300 - img_loss: 0.0157 - txt_loss: 0.0142 - val_loss: 0.0300 - val_img_loss: 0.0157 - val_txt_loss: 0.0143


In [113]:
from sklearn.preprocessing import MinMaxScaler, normalize

norm_x_img_q = normalize(x_img_q)
img_scaler = MinMaxScaler()
img_scaler.fit(norm_x_img_q)
scaled_x_img_q = img_scaler.transform(norm_x_img_q)

norm_x_txt_q = normalize(x_txt_q)
txt_scaler = MinMaxScaler()
txt_scaler.fit(norm_x_txt_q)
scaled_x_txt_q = txt_scaler.transform(norm_x_txt_q)


In [114]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_train, y_test = train_test_split(scaled_x_img_q, scaled_x_txt_q, y_q, test_size=0.2, random_state=42)

In [115]:
print(max([max(e) for e in x_img_train]))

1.0


In [137]:
train_e_relu = img_encoder.predict([x_img_train, x_txt_train])
model = get_model()
model.fit(train_e_relu, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 14s 364us/sample - loss: 2.8712 - accuracy: 0.2620 - val_loss: 2.3990 - val_accuracy: 0.3687
Epoch 2/20
38221/38221 [==============================] - 11s 290us/sample - loss: 2.4943 - accuracy: 0.3447 - val_loss: 2.2984 - val_accuracy: 0.3946
Epoch 3/20
38221/38221 [==============================] - 10s 272us/sample - loss: 2.4074 - accuracy: 0.3650 - val_loss: 2.2788 - val_accuracy: 0.3918
Epoch 4/20
38221/38221 [==============================] - 11s 292us/sample - loss: 2.3635 - accuracy: 0.3766 - val_loss: 2.2361 - val_accuracy: 0.4033
Epoch 5/20
38221/38221 [==============================] - 11s 296us/sample - loss: 2.3246 - accuracy: 0.3853 - val_loss: 2.2213 - val_accuracy: 0.4040
Epoch 6/20
38221/38221 [==============================] - 11s 289us/sample - loss: 2.2940 - accuracy: 0.3932 - val_loss: 2.2045 - val_accuracy: 0.4121
Epoch 7/20
38221/38221 [=====================

In [138]:
def get_model_13():
    inp = Input(shape=(512,))
    x = Dense(512, activation='relu')(inp)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(50, activation='softmax')(x)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [139]:
model = get_model_13()
model.fit(train_e_relu, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 25s 662us/sample - loss: 2.9389 - accuracy: 0.2389 - val_loss: 2.4804 - val_accuracy: 0.3464
Epoch 2/20
38221/38221 [==============================] - 20s 521us/sample - loss: 2.5918 - accuracy: 0.3172 - val_loss: 2.3742 - val_accuracy: 0.3753
Epoch 3/20
38221/38221 [==============================] - 20s 535us/sample - loss: 2.5115 - accuracy: 0.3406 - val_loss: 2.3303 - val_accuracy: 0.3866
Epoch 4/20
38221/38221 [==============================] - 18s 459us/sample - loss: 2.4546 - accuracy: 0.3560 - val_loss: 2.2991 - val_accuracy: 0.3885
Epoch 5/20
38221/38221 [==============================] - 17s 453us/sample - loss: 2.4147 - accuracy: 0.3662 - val_loss: 2.2717 - val_accuracy: 0.3939
Epoch 6/20
38221/38221 [==============================] - 19s 498us/sample - loss: 2.3833 - accuracy: 0.3739 - val_loss: 2.2747 - val_accuracy: 0.3972
Epoch 7/20
38221/38221 [=====================

In [142]:
def get_combined_model(encoder, model):
    inp_img = Input(shape=(IMG_LEN,))
    inp_txt = Input(shape=(TXT_LEN,))
    
    output_e = encoder([inp_img, inp_txt])
    output_m = model(output_e)
    
    return Model([inp_img, inp_txt], output_m)

In [147]:
combined_model = get_combined_model(img_encoder, model)
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [148]:
combined_model.evaluate([x_img_test, x_txt_test], y_test, verbose=0)

[2.2489103235699446, 0.403598]

In [150]:
combined_model.fit([x_img_train, x_txt_train], y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 16s 413us/sample - loss: 2.7878 - accuracy: 0.2865 - val_loss: 2.4970 - val_accuracy: 0.3645
Epoch 2/20
38221/38221 [==============================] - 15s 405us/sample - loss: 2.6210 - accuracy: 0.3245 - val_loss: 2.4475 - val_accuracy: 0.3546
Epoch 3/20
38221/38221 [==============================] - 17s 435us/sample - loss: 2.5425 - accuracy: 0.3414 - val_loss: 2.4003 - val_accuracy: 0.3744
Epoch 4/20
38221/38221 [==============================] - 20s 514us/sample - loss: 2.4934 - accuracy: 0.3531 - val_loss: 2.3637 - val_accuracy: 0.3958
Epoch 5/20
38221/38221 [==============================] - 16s 412us/sample - loss: 2.4510 - accuracy: 0.3664 - val_loss: 2.3418 - val_accuracy: 0.3951
Epoch 6/20
38221/38221 [==============================] - 15s 386us/sample - loss: 2.4306 - accuracy: 0.3710 - val_loss: 2.3179 - val_accuracy: 0.3932
Epoch 7/20
38221/38221 [=====================

In [151]:
train_e_relu_1 = relu_encoder_1.predict([x_img_train, x_txt_train])
model = get_model()
model.fit(train_e_relu_1, y_train, epochs=20, validation_split=0.1)

Train on 38221 samples, validate on 4247 samples
Epoch 1/20
38221/38221 [==============================] - 8s 213us/sample - loss: 2.8639 - accuracy: 0.2604 - val_loss: 2.4127 - val_accuracy: 0.3659
Epoch 2/20
38221/38221 [==============================] - 8s 202us/sample - loss: 2.4906 - accuracy: 0.3480 - val_loss: 2.2960 - val_accuracy: 0.3859
Epoch 3/20
38221/38221 [==============================] - 8s 209us/sample - loss: 2.4077 - accuracy: 0.3677 - val_loss: 2.2594 - val_accuracy: 0.4036
Epoch 4/20
38221/38221 [==============================] - 7s 187us/sample - loss: 2.3527 - accuracy: 0.3815 - val_loss: 2.2357 - val_accuracy: 0.4005
Epoch 5/20
38221/38221 [==============================] - 8s 200us/sample - loss: 2.3235 - accuracy: 0.3899 - val_loss: 2.2165 - val_accuracy: 0.4132
Epoch 6/20
38221/38221 [==============================] - 7s 188us/sample - loss: 2.2869 - accuracy: 0.3956 - val_loss: 2.2158 - val_accuracy: 0.4092
Epoch 7/20
38221/38221 [===========================